In [155]:
import pandas as pd

rome40 = pd.read_csv(filepath_or_buffer = '../data.gouv/rome40.csv',
                           sep = ';',
                           header=None, names=['A', 'B', 'C', 'D', 'E'])  # numéro de la colonne qui indexe les entrées

rome40.info()
rome40.describe()
rome40.isna().any(axis = 1)
rome40.isnull().sum()
rome40 = rome40.loc[rome40['A'] != ' ']
rome40 = rome40.loc[rome40['B'] != ' ']
rome40 = rome40.loc[rome40['C'] != ' ']
rome40 = rome40.loc[rome40['D'] != ' ']
rome40['code'] = rome40['A'].astype(str) + rome40['B'].astype(str) + rome40['C'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13287 entries, 0 to 13286
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       13287 non-null  object
 1   B       13287 non-null  object
 2   C       13287 non-null  object
 3   D       13287 non-null  object
 4   E       13287 non-null  object
dtypes: object(5)
memory usage: 519.2+ KB


In [156]:
rome = rome40[['code','D']]
rome["libelle"] = rome["D"]
rome.drop("D", axis=1, inplace=True)
rome = rome.drop_duplicates(keep='first')

rome.head(10)

/var/folders/c2/wwmb5wgx0nq_syd01vp_bt500000gn/T/ipykernel_21243/1709500014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rome["libelle"] = rome["D"]
/var/folders/c2/wwmb5wgx0nq_syd01vp_bt500000gn/T/ipykernel_21243/1709500014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rome.drop("D", axis=1, inplace=True)


,code,libelle
2,A1101,Conducteur / Conductrice d'engins agricoles
3,A1101,Chauffeur / Chauffeuse de machines agricoles
4,A1101,Conducteur / Conductrice d'automoteur de récolte
6,A1101,Conducteur / Conductrice d'engins d'exploitati...
7,A1101,Conducteur / Conductrice d'épareuse
8,A1101,Conducteur / Conductrice de machines à vendanger
9,A1101,Conducteur / Conductrice de matériels de semis
10,A1101,Conducteur / Conductrice de pulvérisateur
11,A1101,Conducteur / Conductrice de tracteur
12,A1101,Conducteur / Conductrice de tracteur enjambeur


In [157]:
print(rome40.head(5))

   A   B   C                                                  D       E   code
2  A  11  01        Conducteur / Conductrice d'engins agricoles       6  A1101
3  A  11  01       Chauffeur / Chauffeuse de machines agricoles   11987  A1101
4  A  11  01   Conducteur / Conductrice d'automoteur de récolte   38874  A1101
5  A  11  01        Conducteur / Conductrice d'engins agricoles  404030  A1101
6  A  11  01  Conducteur / Conductrice d'engins d'exploitati...   13254  A1101


In [158]:
# DIM_ROMECODES

code_rome = rome['code']
code_rome = code_rome.drop_duplicates(keep='first')

# code.to_csv('../data.gouv/rome40-list.csv', encoding='utf-8', header=False, index=False)
code_rome.to_csv('../snowflake/DBT/Projet_DBT/seeds/DIM_ROMECODES.csv', encoding='utf-8', header=False, index=False)

code_rome.head(10)

2     A1101
17    A1102
31    A1201
38    A1202
50    A1203
60    A1204
78    A1205
85    A1206
89    A1207
92    A1208
Name: code, dtype: object

In [159]:
# DIM_DOMAINES

domaines = pd.read_csv(filepath_or_buffer = '../data/france_travail_domaines.csv', sep = ',')
domaines.drop(domaines.columns[0], axis=1, inplace=True)
domaines.info()


domaines.to_csv('../snowflake/DBT/Projet_DBT/seeds/DIM_DOMAINES.csv', encoding='utf-8', header=False, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   code     110 non-null    object
 1   libelle  110 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [160]:
# DIM_APPELLATIONS

appellations = pd.read_csv(filepath_or_buffer = '../data/france_travail_appellations.csv',
                           sep = ',')
appellations.info()
appellations.head()
appellations.drop(appellations.columns[0], axis=1, inplace=True)
appellations.head()

appellations.to_csv('../snowflake/DBT/Projet_DBT/seeds/DIM_APPELLATIONS.csv', encoding='utf-8', header=False, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12107 entries, 0 to 12106
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  12107 non-null  int64 
 1   code        12107 non-null  int64 
 2   libelle     12107 non-null  object
dtypes: int64(2), object(1)
memory usage: 283.9+ KB


In [161]:
# DIM_METIERS

metiers = rome.merge(right=appellations, on='libelle', how='left')
metiers.dropna(axis = 0, how="all", subset = ['code_y'], inplace=True)

columns = {'code_x': 'code_rome', 'code_y': 'code_appellation'}
metiers = metiers.rename(columns, axis = 1)

types = {'code_appellation': 'int'}
metiers = metiers.astype(types)



import re

def get_first_three_chars(input_string):
    """
    Extrait les 3 premiers caractères d'une chaîne.
    
    Args:
        input_string (str): La chaîne d'entrée
        
    Returns:
        str: Les 3 premiers caractères
    """
    pattern = r'^(.{3})'
    match = re.search(pattern, input_string)
    
    if match:
        return match.group(1)
    else:
        return None

metiers["code_domaine"] = metiers["code_rome"].apply(get_first_three_chars)

metiers.head()
metiers.info()
metiers.to_csv('../snowflake/DBT/Projet_DBT/seeds/RAW_METIERS.csv', encoding='utf-8', header=True, index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 12096 entries, 0 to 12107
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   code_rome         12096 non-null  object
 1   libelle           12096 non-null  object
 2   code_appellation  12096 non-null  int64 
 3   code_domaine      12096 non-null  object
dtypes: int64(1), object(3)
memory usage: 472.5+ KB
